# MODELO CLUSTER - WINE QUALITY

Essa análise tem como objetivo a análise do dataset de vinhos com o objetivo de identificar os seus tipos com base em clusters usando algoritmos como KNN e K-Means por exemplo.

Anteriormente já fiz uma análise com o mesmo dataset, mas no caso foi beseado em uma análise com classificação tipo supervisionada. Agora será feito uma análise com aprendizagem não supervisionada, ou seja, vamos identificar os tipos de vinhos sem dar pistas ao modelo com um dataset de treino. 